In [1]:
import pandas as pd
import numpy as np

In [2]:
transactions = pd.read_pickle("pickles/transactions")
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


### Groupage par client

In [32]:
transactions_by_customer = transactions.groupby("customer_id", as_index = False).agg(
    date_min = ("t_dat", "min"),
    date_max = ("t_dat", "max"),
    total = ("price", "sum"),
    unique_articles = ("article_id", "nunique"),
    count = ("article_id", "count")
)

transactions_by_customer.to_pickle('pickles/transactions_by_customer')
transactions_by_customer.head()

,customer_id,date_min,date_max,total,unique_articles,count
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,2020-09-05,0.648983,19,21
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2018-09-21,2020-07-08,2.601932,64,86
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,2018-09-20,2020-09-15,0.704780,14,18
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,2019-06-09,2019-06-09,0.060983,2,2
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,2018-10-12,2020-08-12,0.469695,12,13


-> On est sur des périodes de plusieurs années, avec un nombre important d'articles achetés, et parfois des articles qui ont été achetés plusieurs fois.

#### Groupage par article

In [31]:
transactions_by_article = transactions.groupby("article_id", as_index=False).agg(
    date_min = ("t_dat", "min"),
    date_max = ("t_dat", "max"),
    unique_customers = ("customer_id", "nunique"),
    count = ("customer_id", "count")
)

transactions_by_article.to_pickle('pickles/transactions_by_article')
transactions_by_article.head()

,article_id,date_min,date_max,unique_customers,count
0,108775015,2018-09-20,2020-07-22,6885,10841
1,108775044,2018-09-20,2020-09-20,5284,7250
2,108775051,2018-09-20,2019-06-28,168,215
3,110065001,2018-09-20,2020-08-02,945,1044
4,110065002,2018-09-20,2020-08-05,460,539


### Groupage par article et client

In [26]:
transaction_multiple_purchases = transactions.groupby(['article_id', 'customer_id'], as_index=False).agg(
    count = ('article_id', 'count')
)

transaction_multiple_purchases.to_pickle('pickles/transaction_multiple_purchases')
transaction_multiple_purchases.head()

,article_id,customer_id,count
0,108775015,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,3
1,108775015,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,2
2,108775015,001ba9e81e13ce12a2585d9ebde923fe74429e9e12ea59...,1
3,108775015,0022a721371d5949d174ecba60346d89a9d6c08c0fba4f...,2
4,108775015,002323971cbd38fad4512d5114676e5e17eb262db02320...,1


In [34]:
transaction_multiple_purchases.sort_values('count', ascending = False)

,article_id,customer_id,count
9483089,678342001,d00063b94dcb1342869d4994844a2742b5d62927f36843...,570
6513002,629420001,94665b46e194622ccdbcadc0170f13a2f8ede1ff6d057d...,199
1951536,507909001,61da44a2758206d5701771f4315637b40c8321b5111916...,188
3942848,570002001,ef38ec0f0cb29ee8bbb87efc82fd16f4b99127e3eeefe6...,170
10808357,688558002,5cba04ed9a3759bc02a8a9e01efccc07ce76c35c1a70dc...,166
...,...,...,...
9726398,679854001,bf6ceced97c6963732ce055771986ee076085dcb407f5c...,1
9726397,679854001,bf67c7e813e4739630bd2a9dcde480b0bf4c17bfa94a89...,1
9726396,679854001,bf6157f2516df9d8cc471e59146c83b25487b61c9e1c45...,1
9726395,679854001,bf5c5a23ea6c403d5fc9996bb3c7f9f82866b29fbba01f...,1


-> Certains articles ont été commandés plus de 150 fois par le même client (?)<br>
À déterminer donc de quel type d'article / client il s'agit

In [45]:
articles = pd.read_pickle('pickles/articles')

### À faire

-  Voir quel(s) type(s) de clients achètent plus de 150 fois le même article et dans quel mesure on doit les inclure dans le modèle
- Faire un histogramme du nombre d'achat maximum d'un article par un même client
- Faire un histogramme du nombre d'achat par client
- Faire un histogramme du nombre de clients uniques par article

- Ajouter une colonne transaction précédente / transaction suivante
- Monter un tableau avec les séries (sur 7 jours ?)